In [1]:
from Naver_Blog_Crawl import *
from multiprocessing_on_dill import Pool, cpu_count
from functools import partial
# from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
import os
import lxml

In [2]:
with open("search_keywords.pkl", "rb") as f:
    search_keywords = pickle.load(f)

In [3]:
np.random.shuffle(search_keywords)

In [4]:
num_cores = 40
search_keywords_split = np.array_split(search_keywords, num_cores)

In [5]:
def collect_blog(keywords):
    naver_blog_crawl = NaverBlogCrawl()
    return naver_blog_crawl.collect_blog(keywords, keywords_start_idx = 0, num_blogs = 100)

In [8]:
blogId = "2hailin"
logNoList = "222658923116"
params_dict = {"blogId" : blogId,
                "logNoList" : logNoList,
                "logType" : "mylog"}
base_url = "https://blog.naver.com/BlogTagListInfo.naver"
rq = requests_retry_session().get(base_url, params = params_dict)
rq.json()

In [ ]:
with Pool(num_cores) as p:
    results_async=p.map_async(collect_blog, search_keywords_split)
    results_async.get()

In [12]:
temp = {"A" : 123}
temp2 = {"B" : 223}
temp.update(temp2)
temp

{'A': 123, 'B': 223}

In [ ]:
naver_blog_crawl = NaverBlogCrawl()
result = naver_blog_crawl.collect_blog(search_keywords[:4], keywords_start_idx = 0, num_blogs= 1)
pd.DataFrame(result).to_excel("temp.xlsx")

In [ ]:
naver_blog_crawl = NaverBlogCrawl() # 23.8
result = naver_blog_crawl.collect_blog("LG스탠바이미")

In [13]:
def merge_dict(org_dict:dict, new_dict:dict, type:str = "full"):
    if type == "full":
        keys = pd.unique(list(org_dict.keys()) + list(new_dict.keys()))
    elif type == "left":
        keys = list(org_dict.keys())
    elif type == "right":
        keys = list(new_dict.keys())


    for key in keys:
        if isinstance(new_dict[key], list):
            org_dict[key].extend(new_dict[key])
        else:
            org_dict[key].append(new_dict[key])
    
    return org_dict

In [15]:
result_dict = defaultdict(list)
blog_dicts = os.listdir("./blog_dict")
for x in blog_dicts:
    with open(f"./blog_dict/{x}", "rb") as f:
        blog_dict = pickle.load(f)
        result_dict = merge_dict(result_dict, blog_dict)

In [17]:
pd.DataFrame(result_dict).to_parquet("blog_data_220312.parquet", index = False)

In [19]:
pd.DataFrame(result_dict).to_excel("blog_data_220312.xlsx", index = False, encoding = "CP949")

In [6]:
blog_rq = requests.get("https://blog.naver.com/PostView.naver?blogId=kkalgid1&logNo=221362946932&from=search&redirect=Log&widgetTypeCall=true&directAccess=false")
blog_dom = etree.HTML(blog_rq.text)

In [7]:
blog_body = blog_dom.find(".//div[@class='se_component_wrap sect_dsc __se_component_area']")

In [8]:
blog_img_txt = blog_body.xpath(".//div[re:match(@class, 'se_component se_(paragraph|image) default')]", 
                                       namespaces={"re": "http://exslt.org/regular-expressions"})        

In [26]:
x = blog_img_txt[8]
re.search(r"se_paragraph", x.attrib["class"]) is not None

False

In [27]:
x.text

'\n\t\t\t'